In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
%matplotlib inline

In [2]:
df = pd.read_csv("data/ds6_train.tsv",sep="\t")

In [3]:
df

,ham,THANX 4 PUTTIN DA FONE DOWN ON ME!!
0,ham,So how are you really. What are you up to. How...
1,ham,Joy's father is John. Then John is the NAME of...
2,ham,"Almost there, see u in a sec"
3,ham,Yes baby! We can study all the positions of th...
4,ham,Great! So what attracts you to the brothas?
...,...,...
4451,ham,Was it something u ate?
4452,ham,I dont know why she.s not getting your messages
4453,ham,Ü neva tell me how i noe... I'm not at home in...
4454,ham,Why don't you go tell your friend you're not s...


In [4]:
df[df["ham"] == "ham"].count()

ham                                    3845
THANX 4 PUTTIN DA FONE DOWN ON ME!!    3845
dtype: int64

In [5]:
spam = df[df["ham"] == "spam"].count()[0]
ham = df[df["ham"] == "ham"].count()[0]

In [6]:
spam,ham

(611, 3845)

In [7]:
def get_words(message):
    li = message.lower().split()
    return li

In [8]:
temp = get_words("Hello I am Sagar Patil")
temp

['hello', 'i', 'am', 'sagar', 'patil']

In [9]:
def create_dict(message_list):
    di = {}
    li = [word for message in message_list for word in get_words(message)]

    for i in li:
        di[i] = li.count(i)
    
    di2 = []
    for i in di:
        if di[i] >= 5:
            di2 += [i]
    di3 = {}
    for i,count in enumerate(di2):
        di3[count] = i + 1
        
    return di3

In [10]:
train_emails = list(df["THANX 4 PUTTIN DA FONE DOWN ON ME!!"])
di = create_dict(train_emails)

In [11]:
di

{'so': 1,
 'how': 2,
 'are': 3,
 'you': 4,
 'what': 5,
 'up': 6,
 'to.': 7,
 "how's": 8,
 'the': 9,
 'and': 10,
 "joy's": 11,
 'father': 12,
 'is': 13,
 'then': 14,
 'john': 15,
 'name': 16,
 'of': 17,
 'almost': 18,
 'there,': 19,
 'see': 20,
 'u': 21,
 'in': 22,
 'a': 23,
 'yes': 24,
 'baby!': 25,
 'we': 26,
 'can': 27,
 'study': 28,
 'all': 29,
 'great!': 30,
 'to': 31,
 'dont': 32,
 'use': 33,
 'any': 34,
 'it': 35,
 'shall': 36,
 'be': 37,
 'fine.': 38,
 'i': 39,
 'have': 40,
 'now.': 41,
 'will': 42,
 'dis': 43,
 'world': 44,
 'only': 45,
 'thing': 46,
 'which': 47,
 'with': 48,
 'n': 49,
 'lot': 50,
 'this': 51,
 'my': 52,
 'number': 53,
 'by': 54,
 'hi': 55,
 'good': 56,
 'wish': 57,
 'd': 58,
 'soon': 59,
 'get': 60,
 'hey': 61,
 'r': 62,
 'u?': 63,
 'hope': 64,
 'well': 65,
 'me': 66,
 'an': 67,
 'del': 68,
 'again': 69,
 'long': 70,
 'time': 71,
 'no': 72,
 'give': 73,
 'call': 74,
 'from': 75,
 'tell': 76,
 'your': 77,
 'friends': 78,
 'plan': 79,
 'do': 80,
 'on': 81,
 'va

In [12]:
def transform(message_list,di):
    m,n = len(message_list),len(di)
    x = np.zeros((m,n))
    for row,message in enumerate(message_list):
        for col,word in enumerate(get_words(message)):
            if word in di:
                x[row][col] = di[word]
    return x

In [13]:
train_emails = list(df["THANX 4 PUTTIN DA FONE DOWN ON ME!!"])
x = transform(train_emails,di)

In [14]:
x.shape

(4456, 1757)

In [15]:
phi = spam / (ham + spam)
phi

0.1371184919210054

In [16]:
def getY(training_op):
    y = np.zeros(len(training_op))
    for index,op in enumerate(training_op):
        if op == "spam":
            y[index] = 1
    return y

In [17]:
y = getY(list(df["ham"]))

In [18]:
x1 = []
x0 = []
for index,i in enumerate(y):
    if i == 1:
        x1 += [x[index]]
    else:
        x0 += [x[index]]
x1 = np.array(x1)
x0 = np.array(x0)

In [19]:
x0.shape,x1.shape

((3845, 1757), (611, 1757))

In [20]:
phi_y0 = (1 + x0.sum(axis=0)) / (x0.shape[1] + np.sum(x0))
phi_y1 = (1 + x1.sum(axis=0)) / (x1.shape[1] + np.sum(x1))
phi_y = spam / (spam + ham)

In [21]:
phi_y1.shape,phi_y0.shape

((1757,), (1757,))

In [22]:
phi_y1

array([4.37814266e-02, 3.47447016e-02, 3.71614745e-02, ...,
       2.00945610e-07, 2.00945610e-07, 2.00945610e-07])

In [23]:
op = x @ (np.log(phi_y1) - np.log(phi_y0)) + np.log(phi_y / (1 - phi_y)) >=0

In [24]:
op2 = []
for i in op:
    if i == True:
        op2 += ["spam"]
    else:
        op2 += ["ham"]

In [25]:
df["y predicted"] = op2

In [26]:
y[0],op[0]

(0.0, False)

In [27]:
df_test = pd.read_csv("data/ds6_test.tsv",sep="\t")

In [28]:
df_test

,ham,Ok... The theory test? when are ü going to book? I think it's on 21 may. Coz thought wanna go out with jiayin. But she isnt free
0,ham,Can do lor...
1,ham,Lol ok. I'll snatch her purse too.
2,ham,K and you're sure I don't have to have consent...
3,ham,Yeah that's the impression I got
4,ham,Yeah but which is worse for i
...,...,...
552,ham,Do you know where my lab goggles went
553,ham,Bring it if you got it
554,ham,Meet after lunch la...
555,ham,"Tee hee. Off to lecture, cheery bye bye."


In [29]:
test_spam = df_test[df_test["ham"] == "spam"].count()[0]
test_ham = df_test[df_test["ham"] == "ham"].count()[0]

In [30]:
test_spam,test_ham

(67, 490)

In [31]:
x_test = transform(list(df_test["Ok... The theory test? when are ü going to book? I think it's on 21 may. Coz thought wanna go out with jiayin. But she isnt free"]),di)

In [32]:
x_test

array([[  27.,   80.,  672., ...,    0.,    0.,    0.],
       [ 596.,  531.,  125., ...,    0.,    0.,    0.],
       [ 868.,   10.,  404., ...,    0.,    0.,    0.],
       ...,
       [1010.,  656., 1088., ...,    0.,    0.,    0.],
       [   0.,    0.,  340., ...,    0.,    0.,    0.],
       [   0.,    0.,   39., ...,    0.,    0.,    0.]])

In [33]:
test_op = x_test @ (np.log(phi_y1) - np.log(phi_y0)) + np.log(phi_y / (1 - phi_y)) >=0

In [34]:
test_op2 = []
for i in test_op:
    if i == True:
        test_op2 += ["spam"]
    else:
        test_op2 += ["ham"]

In [35]:
print(len([i for i in test_op2 if i == "spam"]))

146


In [36]:
df_test["y_predicted"] = test_op2

In [37]:
df_test

,ham,Ok... The theory test? when are ü going to book? I think it's on 21 may. Coz thought wanna go out with jiayin. But she isnt free,y_predicted
0,ham,Can do lor...,ham
1,ham,Lol ok. I'll snatch her purse too.,ham
2,ham,K and you're sure I don't have to have consent...,ham
3,ham,Yeah that's the impression I got,ham
4,ham,Yeah but which is worse for i,ham
...,...,...,...
552,ham,Do you know where my lab goggles went,ham
553,ham,Bring it if you got it,ham
554,ham,Meet after lunch la...,ham
555,ham,"Tee hee. Off to lecture, cheery bye bye.",ham


In [38]:
y_pred = getY(op2)
error = 0
for index,i in enumerate(y):
    if i != y_pred[index]:
        error += 1
print("Percentage miss classified examples on Training set = {:.2f}%".format(error / df.shape[0] * 100))

Percentage miss classified examples on Training set = 21.84%


In [39]:
correct = 0
for index,i in enumerate(y):
    if i == 1 and op2[index] == "spam":
        correct += 1
print("Percentage correct spam classification on Training set= {:.2f}%".format(correct / spam * 100))

Percentage correct spam classification on Training set= 78.07%


In [40]:
y_pred = getY(test_op2)
error = 0
for index,i in enumerate(getY(list(df_test["ham"]))):
    if i != y_pred[index]:
        error += 1
print("Percentage miss classified examples on Test set = {:.2f}%".format(error / df_test.shape[0] * 100))

Percentage miss classified examples on Test set = 19.57%


In [41]:
correct = 0
for index,i in enumerate(getY(list(df_test["ham"]))):
    if i == 1 and test_op2[index] == "spam":
        correct += 1
print("Percentage correct spam classification on Test set  = {:.2f}%".format(correct / test_spam * 100))

Percentage correct spam classification on Test set  = 77.61%
